**Chapter 1.1 – Our first machine learning model**

*This introductory case is not in the book*. 

***Content***
1. The problem we want to solve.
2. Get the data.
3. Discover and visualize the data.
4. Select a model and train it.
5. Use the model
6. Exercise

<table align="left">
  <td>
    <a href="https://colab.research.google.com/HOGENT-ML/course/machinelearning/blob/main/130-first_machine_learning_project_solution.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>
  </td>
</table>

# 0. Setup Python

First, let's import a few common modules and ensure MatplotLib plots figures inline. We also check that Python 3.5 or later is installed (although Python 2.x may work, it is deprecated so we strongly recommend you use Python 3 instead), as well as Scikit-Learn ≥0.20.

In [6]:
# Python ≥3.5 is required
import sys
assert sys.version_info >= (3, 5)

# Scikit-Learn ≥0.20 is required
import sklearn
assert sklearn.__version__ >= "0.20"

# Common imports
import numpy as np
import pandas as pd
import os

# To plot pretty figures
%matplotlib inline
import matplotlib as mpl
import matplotlib.pyplot as plt


# 1. The problem we want to solve
Predict future temperatures in Brussels in spring based on temperature registrations since 1883.

Before you continue, ask yourself: 
- What are the instances? 
- Which techniques to use? 
    - reinforcement learning?
    - (un)supervised learning? 
    - classification/regression? 
    - batch/online learning? 




Answers: 
- instances = years
- techniques
    - supervised learning: temperature = label
    - regression: predicting a number
    - univariate regression: predicting single value per year
    - batch learning: no continuous flow of new data

# 2. Get the Data

## Download the Data

Governments and major cities provides lots op open data that can be freely used for analyses:
- data.vlaanderen.be: Flemish government
- data.gov.be: Belgian government
- data.gent.be: city of Gent
- ...

We find average seasonal temperatures in Ukkel (Brussels) at https://www.vlaanderen.be/DataCatalogRecord/72ff6930-87e4-38c4-9a27-d3ddda385637. 

The download link for the spring data (as a csv file) is:
https://www.vmm.be/klimaat/seizoensgemiddelde-temperatuur/lente-temp/@@download/csv

Download the file and place it in a subdirectory "datasets" from directory where this notebook resides. 

Read and explore the dataset in a pandas dataframe:

In [8]:
csv_path = '../datasets/lente temp.csv'
temperatures = pd.read_csv(csv_path)
temperatures.head(10)


,jaar,95 %-betrouwbaarheidsinterval van punten op de trendlijn,gemiddelde van de gemeten waarden,trendlijn
0,1833,"(8.413, 7.197)",8.3,7.805
1,1834,"(8.383, 7.239)",9.0,7.811
2,1835,"(8.356, 7.278)",7.7,7.817
3,1836,"(8.333, 7.313)",8.2,7.823
4,1837,"(8.311, 7.347)",4.9,7.829
5,1838,"(8.292, 7.378)",7.5,7.835
6,1839,"(8.277, 7.407)",6.6,7.842
7,1840,"(8.265, 7.433)",7.7,7.849
8,1841,"(8.254, 7.458)",10.5,7.856
9,1842,"(8.245, 7.481)",8.3,7.863


We need only the column "jaar" and "gemiddelde van de gemeten waarden". 

In [4]:
temperatures = temperatures[['jaar','gemiddelde van de gemeten waarden']]

We rename the columns for easier use.

In [5]:
temperatures = temperatures.rename(columns={"jaar": "year", "gemiddelde van de gemeten waarden": "temp"})

## Take a Quick Look at the Data Structure

In [6]:
temperatures.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 190 entries, 0 to 189
Data columns (total 2 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   year    190 non-null    int64  
 1   temp    190 non-null    float64
dtypes: float64(1), int64(1)
memory usage: 3.1 KB


Observations: 
- 190 years in the dataset (1883-2022)
- no years or temperatues are missing
- all temp values are numeric --> clean data

# 3. Discover and Visualize the Data to Gain Insights

In [7]:
temperatures.head()

,year,temp
0,1833,8.3
1,1834,9.0
2,1835,7.7
3,1836,8.2
4,1837,4.9


In [8]:
temperatures.tail()

,year,temp
185,2018,11.6
186,2019,10.5
187,2020,11.3
188,2021,8.8
189,2022,11.3


The describe() method shows a summary of
the numerical attributes

In [9]:
temperatures.describe()

,year,temp
count,190.000000,190.000000
mean,1927.500000,8.870000
std,54.992424,1.289803
min,1833.000000,4.900000
25%,1880.250000,8.000000
50%,1927.500000,8.800000
75%,1974.750000,9.700000
max,2022.000000,12.300000


## Plot the data in a line chart

Text(0, 0.5, 'Average Spring Temperature in Brussels (°C)')

In [10]:
plt.subplots(figsize=(15,5))
plt.plot(temperatures['year'],temperatures['temp'],'b.')
plt.xlabel("Year")
plt.ylabel("Average Spring Temperature in Brussels (°C)")

# 4. Select and Train a Model

We assume their is a linear relationship between the year and the temperature. We can use Linear Regression to find the optimal straight line through the data points. 

Performing Linear Regression using Scikit-Learn is simple:

In [11]:
from sklearn.linear_model import LinearRegression

# create an instance of the linear regression class
lin_reg = LinearRegression()  

# Fit the model to the data: calculate the parameters of the data using the appropriate mathematical algorithms. This is where the real work happens. 
lin_reg.fit(pd.DataFrame(temperatures['year']), temperatures['temp'])  # LinearRegression.fit() works only with a dataframe as first parameter

# Ask the fitted model for the calculates parameters
lin_reg.intercept_, lin_reg.coef_ 

(-19.563759136060042, array([0.01475163]))

`intercept`: interception of the line with the Y axis   
`coef`: slope or gradient of the curve. In this case it means there is, since 1883, an average yearly increase of 0.015 °C. 

## 5. Use the model

It's easy to visualize the fitted line: just predict two x values

In [12]:
years = np.array([[1800], [2100]])
temps = lin_reg.predict(years)
temps

C:\Users\jcor864\AppData\Roaming\Python\Python39\site-packages\sklearn\base.py:450: UserWarning: X does not have valid feature names, but LinearRegression was fitted with feature names
  warnings.warn(


array([ 6.98916768, 11.41465549])

In [13]:
plt.subplots(figsize=(15,5))
plt.plot(temperatures['year'],temperatures['temp'],'b.',label='Measured Temperature')
plt.plot(years, temps, "r-",label='Linear Model')
plt.xlabel("Year")
plt.ylabel("Average Spring Temperature in Brussels (°C)")
plt.grid()
plt.legend(loc='lower right')

The central objective of the Paris Agreement is its long-term temperature goal to hold global average temperature increase to “well below 2°C above preindustrial levels and pursuing efforts to limit the temperature increase to 1.5°C above pre-industrial levels”.  (see https://climateactiontracker.org/global/cat-thermometer/)

If we assume 1850 is pre-industrial, by how much will, according to our model, the temperature (Brussels, spring) increase by 2100? 

In [14]:
years = np.array([[1850], [2100]])
temps = lin_reg.predict(years)
increase = temps[1] - temps[0]
increase

C:\Users\jcor864\AppData\Roaming\Python\Python39\site-packages\sklearn\base.py:450: UserWarning: X does not have valid feature names, but LinearRegression was fitted with feature names
  warnings.warn(


3.687906502731522

# 6. Exercise

- Download the summer temperatures from the same website as above. 

In [9]:
csv_path = '../datasets/zomer temp.csv'
temperatures = pd.read_csv(csv_path)
temperatures.head(10)

,jaar,95 %-betrouwbaarheidsinterval van punten op de trendlijn,gemiddelde van de gemeten waarden,trendlijn
0,1833,"(16.509, 15.361)",15.1,15.935
1,1834,"(16.476, 15.394)",17.9,15.935
2,1835,"(16.446, 15.422)",16.4,15.934
3,1836,"(16.416, 15.452)",16.0,15.934
4,1837,"(16.391, 15.477)",16.1,15.934
5,1838,"(16.367, 15.501)",15.3,15.934
6,1839,"(16.346, 15.522)",15.9,15.934
7,1840,"(16.327, 15.543)",15.3,15.935
8,1841,"(16.312, 15.56)",13.9,15.936
9,1842,"(16.299, 15.577)",16.9,15.938


- Create a model for the temperatures. Do you get similar results?


In [10]:
temperatures = temperatures[['jaar','gemiddelde van de gemeten waarden']]
temperatures = temperatures.rename(columns={"jaar": "year", "gemiddelde van de gemeten waarden": "temp"})
from sklearn.linear_model import LinearRegression

# create an instance of the linear regression class
lin_reg = LinearRegression()  

# Fit the model to the data: calculate the parameters of the data using the appropriate mathematical algorithms. This is where the real work happens. 
lin_reg.fit(pd.DataFrame(temperatures['year']), temperatures['temp'])  # LinearRegression.fit() works only with a dataframe as first parameter

# Ask the fitted model for the calculates parameters
lin_reg.intercept_, lin_reg.coef_ 

(-6.673032515844593, array([0.01198798]))

- Maybe things are getting even worse since approximately 1950. Find a way to check that. 

In [11]:
temperatures_before_1950 = temperatures[temperatures['year'] < 1950]
# Fit the model to the data: calculate the parameters of the data using the appropriate mathematical algorithms. This is where the real work happens. 
lin_reg1 = LinearRegression()
lin_reg1.fit(pd.DataFrame(temperatures_before_1950['year']), temperatures_before_1950['temp'])  # LinearRegression.fit() works only with a dataframe as first parameter

# Ask the fitted model for the calculates parameters
lin_reg1.intercept_, lin_reg1.coef_ 

(7.796182069764772, array([0.00432795]))

In [12]:
temperatures_as_of_1950 = temperatures[temperatures['year'] >= 1950]
# Fit the model to the data: calculate the parameters of the data using the appropriate mathematical algorithms. This is where the real work happens. 
lin_reg2 = LinearRegression()
lin_reg2.fit(pd.DataFrame(temperatures_as_of_1950['year']), temperatures_as_of_1950['temp'])  # LinearRegression.fit() works only with a dataframe as first parameter

# Ask the fitted model for the calculates parameters
lin_reg2.intercept_, lin_reg2.coef_ 

(-56.348257682339856, array([0.03700703]))

In [13]:
years1 = np.array([[1800], [1949]])
temps1 = lin_reg1.predict(years1)
temps1

c:\Users\jcor864\AppData\Local\Programs\Python\Python310\lib\site-packages\sklearn\base.py:439: UserWarning: X does not have valid feature names, but LinearRegression was fitted with feature names
  warnings.warn(


array([15.58649812, 16.23136318])

In [14]:
years2 = np.array([[1950], [2100]])
temps2 = lin_reg2.predict(years2)
temps2

c:\Users\jcor864\AppData\Local\Programs\Python\Python310\lib\site-packages\sklearn\base.py:439: UserWarning: X does not have valid feature names, but LinearRegression was fitted with feature names
  warnings.warn(


array([15.81545946, 21.36651462])

In [15]:
plt.subplots(figsize=(15,5))
plt.plot(temperatures['year'],temperatures['temp'],'b.',label='Measured Temperature')
plt.plot(years1, temps1, "r-",label='Linear Model')
plt.plot(years2, temps2, "r-",label='Linear Model')
plt.xlabel("Year")
plt.ylabel("Average Spring Temperature in Brussels (°C)")
plt.grid()
plt.legend(loc='lower right')